### The code for filtering the national urban heritages mentioned in the listing description

PS: Package 'pkuseg' cannot be used under python 3.8 environment.

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime as datetime
import re

airbnb = pd.read_csv("cleaned data airbnb.csv", encoding='utf-8') #read the cleaned Airbnb listing data
keywords  = pd.read_csv('keyword.txt', sep="\t", encoding='utf-8')
keywords['keyword'] = [tuple(i.split(",")) for i in keywords['keyword']]
uncn = re.compile(r'[\u4e00-\u9fa5]')

# Sentence Segmentation
l = []
for i in range(len(airbnb)):
    file = airbnb.loc[i,'description']
    l.append(file)
print(len(l))

def cut_sent(para):
    para = re.sub('([。！~ ？\?])([^”’])', r"\1\n\2", para)  # single character sentence break
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)  # English ellipsis
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  # Chinese ellipsis
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    para = para.rstrip()  
    return para.split("\n")

sentence = []
for para in l:
    sentence.append(cut_sent(para))
print(len(sentence))
print('Finished sentence segmentation.')

# Word Segmentation
# pkuseg - 'tourism' model + User Defined Dictionary
import pkuseg
from datetime import datetime
def pk_tourism2(sent):
    print(datetime.now())
    seg  = pkuseg.pkuseg(model_name='tourism',user_dict = 'listing.txt' ,postag=True) #load the model
    out_p = []
    for w in sent:
        out_p.append(seg.cut(w))
    return out_p

o = []
for sent in sentence:
    p = pk_tourism2(sent)
    o.append(p)
print(len(o))
print('Finished word segmentation.')

s = [] # An element in this list represents a list of selected words obtained from the description
for oo in o:
    ss = []
    for ooo in oo:
        for oooo in ooo:
            if oooo[1] in ('n','s','j','nr','ns','nt','nz'):
                if '\u4e00' <= oooo[0] and oooo[0] <= '\u9fa5':
                    ss.append(oooo)
    s.append(ss)
print(len(s))
print('Finished selected words.')

airbnb_t = airbnb
print(airbnb_t.shape)
airbnb_t['keyword'] = 0
for j in range(len(s)): 
    s_l = []
    keywords_c = keywords
    for sss in s[j]:
        if len(keywords_c) == 0:
            break
        for n in range(len(keywords_c)):
            if sss[0] in keywords_c.loc[n,'keyword']:
                s_l.append(keywords_c.loc[n,'listing'])
                keywords_c = keywords_c.drop(n)
                keywords_c = keywords_c.reset_index(drop=True)
                break
    if s_l == []:
        s_l = 0
    else:
        s_l = set(s_l) 
    airbnb_t.loc[j,'keyword'] = str(s_l)
    
airbnb_t['keyword'] = [i.replace('{', '') for i in airbnb_t['keyword']]
airbnb_t['keyword'] = [i.replace('}', '') for i in airbnb_t['keyword']]
airbnb_t['keyword'] = [i.replace('\'', '') for i in airbnb_t['keyword']]

for i in range(len(airbnb)):
    l = list(airbnb.loc[i,'keyword'].split(", "))
    l.sort()

    airbnb.loc[i,'keyword'] = str(l)
airbnb['keyword'] = [i.replace('[', '') for i in airbnb['keyword']]
airbnb['keyword'] = [i.replace(']', '') for i in airbnb['keyword']]
airbnb['keyword'] = [i.replace('\'', '') for i in airbnb['keyword']]

airbnb['keyword'] = [i.replace('{', '') for i in airbnb['keyword']]
airbnb['keyword'] = [i.replace('}', '') for i in airbnb['keyword']]
airbnb['keyword'] = [i.replace('\'', '') for i in airbnb['keyword']]
        
airbnb.to_csv("result.csv", index=False, index_label=False) 